In [ ]:
import pandas as pd
import os 
import re

# Clean Data

In [ ]:
# Clean data
import os

folder_path = '/Users/fei/Desktop/Mixer/web_crawler'

dfs = [] # list to hold all dataframes prior to concatenation

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dfs.append(df)

product_data_all = pd.concat(dfs)

print(product_data_all.head())
print(product_data_all.shape)

In [ ]:
product_data_all.isna().sum()

In [ ]:
product_data_all.isna().any(axis=1).sum()

In [ ]:
test = product_data_all.iloc[2, 5]

In [ ]:
test

In [ ]:
# remove unwanted strings

while "\n" in test or "\u200e" in test or "Product information" in test or "Technical Details" in test or "Additional Information" in test or "out of 5 stars" in test:
    if "\n" in test:
        index = test.find("\n")
        test = test[:index] + test[index+1:]
    if "\u200e" in test:
        index1 = test.find("\u200e")
        test = test[:index1] + test[index1+1:]
    if "Product information" in test:
        index2 = test.find("Product information")
        test = test[:index2] + test[index2+20:]
    if "Technical Details" in test:
        index3 = test.find("Technical Details")
        test = test[:index3] + test[index3+18:]
    if "Additional Information" in test:
        index4 = test.find("Additional Information")
        test = test[:index4] + test[index4+23:]
    if "out of 5 stars" in test:
        index5a = test.find("Customer Reviews")
        matches = re.finditer("out of 5 stars", test)
        positions = [match.start() for match in matches]
        index5b = positions[-1]
        test = test[:index5a] + test[index5b+14:]
        
test = test.strip()
test

In [ ]:
raw_terms = test.split("   ")

while "" in raw_terms:
    for term in raw_terms:
        if term == "":
            raw_terms.remove(term)

terms = [term.strip() for term in raw_terms]
terms

In [ ]:
len(terms)

In [ ]:
product_info_cat = {}

indices = list(range(0, len(terms)-1, 2))

for i in indices:
    if i % 2 == 0:
        product_info_cat[terms[i]] = terms[i+1]

for key, value in product_info_cat.items():
    print(f"{key}: {value}")

In [ ]:
# for i in range(len(product_data_all)):
#     if "out of 5 stars" in product_data_all['Rating'][i]:
#         product_data_all['Rating'][i] = product_data_all['Rating'][i][:3]
#     if "ratings" in product_data_all['Number of Ratings'][i]:
#         product_data_all['Number of Ratings'][i] = product_data_all['Number of Ratings'][i][:-8]
#     if "+ bought in past month" in product_data_all['Monthly Purchases'][i]:
#         product_data_all['Monthly Purchases'][i] = product_data_all['Monthly Purchases'][i][:-22]
#     if "K" in product_data_all['Monthly Purchases'][i]:
#         product_data_all['Monthly Purchases'][i] = int(product_data_all['Monthly Purchases'][i][:-1]) * 1000
#     if "$" in product_data_all['Price'][i]:
#         product_data_all['Price'][i] = product_data_all['Price'][i][1:]

# product_data_all['Rating'].astype(float)
# #product_data_all['Number of Ratings'].astype(int)
# product_data_all['Monthly Purchases'].astype(int)
# product_data_all['Price'].astype(float)
# product_data_all.head()